In [ ]:
# default_exp summarization

# Summarization
> Performing summarization within the AdaptNLP library

In [ ]:
#hide
from nbdev.showdoc import *
from fastcore.test import test_eq

In [ ]:
#export
import logging
from typing import List, Dict, Union
from collections import defaultdict

import torch
from torch.utils.data import TensorDataset, DataLoader

from transformers import (
    AutoTokenizer,
    AutoModelForSeq2SeqLM,
    PreTrainedTokenizer,
    PreTrainedModel,
    T5ForConditionalGeneration,
    BartForConditionalGeneration,
)

from adaptnlp.callback import GeneratorCallback
from adaptnlp.model import AdaptiveModel

from fastcore.basics import store_attr
from fastcore.meta import delegates

from fastai_minima.callback.core import Callback, CancelBatchException
from fastai_minima.utils import apply

In [ ]:
#export
logger = logging.getLogger(__name__)

In [ ]:
#export
class TransformersSummarizer(AdaptiveModel):
    """Adaptive model for Transformer's Conditional Generation or Language Models (Transformer's T5 and Bart
    conditiional generation models have a language modeling head)

    Usage:
    ```python
    >>> summarizer = TransformersSummarizer.load("transformers-summarizer-model")
    >>> summarizer.predict(text="Example text", mini_batch_size=32)
    ```

    **Parameters:**

    * **tokenizer** - A tokenizer object from Huggingface's transformers (TODO)and tokenizers
    * **model** - A transformers Conditional Generation (Bart or T5) or Language model
    """

    def __init__(self, tokenizer: PreTrainedTokenizer, model: PreTrainedModel):
        # Load up tokenizer
        self.tokenizer = tokenizer
        
        super().__init__()
        # Sets internal model
        super().set_model(model)

        # Set inputs to come in as `dict`
        super().set_as_dict(True)

    @classmethod
    def load(cls, model_name_or_path: str) -> AdaptiveModel:
        """Class method for loading and constructing this classifier

        * **model_name_or_path** - A key string of one of Transformer's pre-trained Summarizer Model
        """
        tokenizer = AutoTokenizer.from_pretrained(model_name_or_path)
        model = AutoModelForSeq2SeqLM.from_pretrained(model_name_or_path)
        summarizer = cls(tokenizer, model)
        return summarizer

    def predict(
        self,
        text: Union[List[str], str],
        mini_batch_size: int = 32,
        num_beams: int = 4,
        min_length: int = 0,
        max_length: int = 128,
        early_stopping: bool = True,
        **kwargs,
    ) -> List[str]:
        """Predict method for running inference using the pre-trained sequence classifier model

        * **text** - String, list of strings, sentences, or list of sentences to run inference on
        * **mini_batch_size** - Mini batch size
        * **num_beams** - Number of beams for beam search. Must be between 1 and infinity. 1 means no beam search.  Default to 4.
        * **min_length** -  The min length of the sequence to be generated. Default to 0
        * **max_length** - The max length of the sequence to be generated. Between min_length and infinity. Default to 128
        * **early_stopping** - if set to True beam search is stopped when at least num_beams sentences finished per batch.
        * **&ast;&ast;kwargs**(Optional) - Optional arguments for the Transformers `PreTrainedModel.generate()` method
        """

        # Make all inputs list
        if isinstance(text, str):
            text = [text]

        # T5 requires 'summarize: ' precursor text for pre-trained summarizer
        if isinstance(self.model, T5ForConditionalGeneration):
            text = [f'summarize: {t}' for t in text]

        dataset = self._tokenize(text)
        dl = DataLoader(dataset, batch_size=mini_batch_size)

        summaries = []

        logger.info(f'Running summarizer on {len(dataset)} text sequences')
        logger.info(f'Batch size = {mini_batch_size}')

        cb = GeneratorCallback(num_beams, min_length, max_length, early_stopping, **kwargs)

        preds,_ = super().get_preds(dl=dl, cbs=[cb])

        preds = apply(lambda x: x.squeeze(0), preds)

        for o in preds:
            summaries.append(
                [
                    self.tokenizer.decode(
                        o,
                        skip_special_tokens=True,
                        clean_up_tokenization_spaces=False,
                    )
                ].pop()
            )

        return summaries

    def _tokenize(self, text: Union[List[str], str]) -> TensorDataset:
        """ Batch tokenizes text and produces a `TensorDataset` with text """

        # Pre-trained Bart summarization model has a max length fo 1024 tokens for input
        if isinstance(self.model, BartForConditionalGeneration):
            tokenized_text = self.tokenizer.batch_encode_plus(
                text,
                return_tensors="pt",
                max_length=1024,
                add_special_tokens=True,
                padding="max_length",
            )
        else:
            tokenized_text = self.tokenizer.batch_encode_plus(
                text,
                return_tensors="pt",
                padding="max_length",
                add_special_tokens=True,
            )

        # Bart doesn't use `token_type_ids`
        dataset = TensorDataset(
            tokenized_text["input_ids"],
            tokenized_text["attention_mask"],
        )

        return dataset

    def train(
        self,
    ):
        raise NotImplementedError

    def evaluate(
        self,
    ):
        raise NotImplementedError

In [ ]:
#export
class EasySummarizer:
    """Summarization Module

    Usage:

    ```python
    >>> summarizer = EasySummarizer()
    >>> summarizer.summarize(text="Summarize this text", model_name_or_path="t5-small")
    ```

    """

    def __init__(self):
        self.summarizers: Dict[AdaptiveModel] = defaultdict(bool)

    def summarize(
        self,
        text: Union[List[str], str],
        model_name_or_path: str = "t5-small",
        mini_batch_size: int = 32,
        num_beams: int = 4,
        min_length: int = 0,
        max_length: int = 128,
        early_stopping: bool = True,
        **kwargs,
    ) -> List[str]:
        """Predict method for running inference using the pre-trained sequence classifier model

        * **text** - String, list of strings, sentences, or list of sentences to run inference on
        * **model_name_or_path** - A String model id or path to a pre-trained model repository or custom trained model directory
        * **mini_batch_size** - Mini batch size
        * **num_beams** - Number of beams for beam search. Must be between 1 and infinity. 1 means no beam search.  Default to 4.
        * **min_length** -  The min length of the sequence to be generated. Default to 0
        * **max_length** - The max length of the sequence to be generated. Between min_length and infinity. Default to 128
        * **early_stopping** - if set to True beam search is stopped when at least num_beams sentences finished per batch.
        * **&ast;&ast;kwargs**(Optional) - Optional arguments for the Transformers `PreTrainedModel.generate()` method
        """
        name = getattr(model_name_or_path, 'name', model_name_or_path)
        if not self.summarizers[name]:
            self.summarizers[name] = TransformersSummarizer.load(
                name
            )

        summarizer = self.summarizers[name]
        return summarizer.predict(
            text=text,
            mini_batch_size=mini_batch_size,
            num_beams=num_beams,
            min_length=min_length,
            max_length=max_length,
            early_stopping=early_stopping,
            **kwargs,
        )

In [ ]:
#hide
# Text from encyclopedia Britannica on Einstein
text = ["""Einstein’s education was disrupted by his father’s repeated failures at business. In 1894, after his company failed to get an important 
          contract to electrify the city of Munich, Hermann Einstein moved to Milan to work with a relative. Einstein was left at a boardinghouse in 
          Munich and expected to finish his education. Alone, miserable, and repelled by the looming prospect of military duty when he turned 16, Einstein 
          ran away six months later and landed on the doorstep of his surprised parents. His parents realized the enormous problems that he faced as a 
          school dropout and draft dodger with no employable skills. His prospects did not look promising.
          Fortunately, Einstein could apply directly to the Eidgenössische Polytechnische Schule (“Swiss Federal Polytechnic School”; in 1911, 
          following expansion in 1909 to full university status, it was renamed the Eidgenössische Technische Hochschule, or “Swiss Federal 
          Institute of Technology”) in Zürich without the equivalent of a high school diploma if he passed its stiff entrance examinations. His marks 
          showed that he excelled in mathematics and physics, but he failed at French, chemistry, and biology. Because of his exceptional math scores, 
          he was allowed into the polytechnic on the condition that he first finish his formal schooling. He went to a special high school run by 
          Jost Winteler in Aarau, Switzerland, and graduated in 1896. He also renounced his German citizenship at that time. (He was stateless until 1901, 
          when he was granted Swiss citizenship.) He became lifelong friends with the Winteler family, with whom he had been boarding. (Winteler’s 
          daughter, Marie, was Einstein’s first love; Einstein’s sister, Maja, would eventually marry Winteler’s son Paul; and his close friend Michele 
          Besso would marry their eldest daughter, Anna.)""",
       """Einstein would write that two “wonders” deeply affected his early years. The first was his encounter with a compass at age five. 
          He was mystified that invisible forces could deflect the needle. This would lead to a lifelong fascination with invisible forces. 
          The second wonder came at age 12 when he discovered a book of geometry, which he devoured, calling it his 'sacred little geometry 
          book'. Einstein became deeply religious at age 12, even composing several songs in praise of God and chanting religious songs on 
          the way to school. This began to change, however, after he read science books that contradicted his religious beliefs. This challenge 
          to established authority left a deep and lasting impression. At the Luitpold Gymnasium, Einstein often felt out of place and victimized 
          by a Prussian-style educational system that seemed to stifle originality and creativity. One teacher even told him that he would 
          never amount to anything."""]

summarizer = EasySummarizer()
summaries = summarizer.summarize(text = text, model_name_or_path="t5-small", mini_batch_size=1, num_beams = 4, min_length=0, max_length=100, early_stopping=True)
test_eq(summaries, ['Hermann Einstein was left at a boardinghouse and expected to finish his education . he ran away six months later and landed on the doorstep of his surprised parents . he could apply directly to the Eidgenössische Polytechnische Schule without the equivalent of a high school diploma .',
 'Einstein was mystified that invisible forces could deflect the needle . the second wonder came at age 12 when he discovered a book of geometry . he became deeply religious at age 12 .'])

In [ ]:
#hide
summaries = summarizer.summarize(text = text, model_name_or_path="facebook/bart-large-cnn", mini_batch_size=1, num_beams = 2, min_length=40, max_length=300, early_stopping=True)
test_eq(summaries, ['Einstein’s education was disrupted by his father’S repeated failures at business. In 1894, after his company failed to get an important contract, Einstein moved to Milan to work with a relative. Einstein was left at a boardinghouse in Munich and expected to finish his education.',
 'Einstein would write that two ‘wonders’ deeply affected his early years. The first was his encounter with a compass at age five. The second wonder came at age 12 when he discovered a book of geometry.'])

In [ ]:
#hide
from adaptnlp.model_hub import HFModelHub
hub = HFModelHub()
models = hub.search_model_by_task('summarization')
model = models[-1]
summaries = summarizer.summarize(text = text, model_name_or_path=model, mini_batch_size=1, num_beams = 2, min_length=40, max_length=300, early_stopping=True)
test_eq(summaries, ['Swiss Federal Poly Polytechnic on the condition thatXtechnic and:”; in 1911, following expansion in 1909 to full university status, it wasX de eventually marry Winteler in Aarau, Switzerland, and graduated in 18X son Paul; and his close friend Michele Besso would marry their marry theirX de marry their their marry, Anna.))esteldest daughterX',
 'would wonder came at this age 12 when de The second wonder came came at age 12Xul early years. The first was his encounter withacompass at age five. He was mys with invisible forces could deflect the needle. This would lead toa lifelong fascination with invisibleX: EinsteinX'])